# Boston Listings

In [1]:
## The following Analysis follows the CRISP-DM process

# **Reading the Dataset**

In [ ]:
# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Loading the Dataset
BostonCalendar = pd.read_csv("Boston/calendar.csv")
BostonListings = pd.read_csv("Boston/listings.csv")
BostonReviews = pd.read_csv("Boston/reviews.csv")

# Business Understanding

## Understanding the insights to be gained from Data

In [ ]:
print (BostonCalendar.head())

In [ ]:
print (BostonReviews.head())

In [ ]:
print (BostonListings.head())

#  Data Understanding

In [ ]:
BostonReviews

In [ ]:
BostonCalendar.columns

In [ ]:
BostonListings.columns

In [ ]:
BostonReviews.columns

In [ ]:
BostonCalendar['available'].value_counts()

In [ ]:
# Trying to check if there is any bias in the reviews due to missing URL's
BostonListings['listing_url'].isnull().sum()

# Most Popular Listings 
## Data Preparation

In [ ]:
# Sorting the listings according to number of reviews to observe the most popular lisitings.
most_reviews = BostonListings[['id','number_of_reviews']].sort_values(by = ['number_of_reviews'],ascending = False)

## Analysis

In [ ]:
most_reviews.head()

## From the table given above we can observe some of most popular lisitings in Boston having the maximum reviews

# __Identifying the most expensive zipcodes__
## Data Preparation

In [ ]:
zipcode_id_df = BostonListings[['zipcode','id']].copy()

In [ ]:
price_id_df = BostonCalendar[['listing_id','price']].copy()

In [ ]:
price_id_df.dropna(axis = 0,inplace = True)

In [ ]:
# Formatting the price to be used a float instead of object
price_id_df['price'] = price_id_df['price'].apply(lambda x: float(x.replace(",","").strip('$')))

In [ ]:
#Calculating the maximum price of a listing in a year 
max_price_df = price_id_df.groupby(by = 'listing_id').max()

In [ ]:
#Calculating the average price of a listing in a year 
mean_price_df = price_id_df.groupby(by = 'listing_id').mean()

In [ ]:
mean_max_merge = max_price_df.join(mean_price_df ,on = 'listing_id',lsuffix = "max",rsuffix = "mean")

In [ ]:
mean_max_merge.head()

In [ ]:
zipcode_id_df.dropna(subset = ['zipcode'], axis = 0,inplace = True)

In [ ]:
zipcode_id_df.columns = ['zipcode','listing_id']

In [ ]:
# Adding the zip codes of each listing
zipcode_price_merge = zipcode_id_df.merge(mean_max_merge,on = 'listing_id')

In [ ]:
zipcode_price_merge.head()

In [ ]:
zipcode_price_merge.sort_values(by = ['zipcode'],inplace = True)

In [ ]:
zipcode_price_merge.drop(columns = ['listing_id'],axis =1,inplace = True)

In [ ]:
zipcode_price_merge.head()

In [ ]:
#Calculating the average prices of any zip code
zipcode_price_mean = zipcode_price_merge.groupby(by = ['zipcode']).mean()

In [ ]:
#Calculating the number of listings in each zip code
zipcode_listings = zipcode_price_merge.groupby(by = ['zipcode']).count()

In [ ]:
df_avg_price = zipcode_price_mean.join(zipcode_listings, on = 'zipcode',rsuffix = "count")

In [ ]:
df_avg_price.head()

## Analysis

In [ ]:

df_avg_sorted = df_avg_price.sort_values(by = ['pricemax','pricemean'],ascending = False)

In [ ]:
df_avg_sorted.head()

In [ ]:
#Since both count columns have same count we can drop one of the variables
df_avg_sorted.drop(labels = ['pricemaxcount'],axis =1,inplace  = True)


In [ ]:
# Below is the table displaying the most expensive zip codes in Boston
df_avg_sorted.head()

### We can see in the table above the top most expensive areas in Boston based on the prices throughout the whole year. The top zip code seems like a single property falling on both zip codes.
### Hence we consider 02210 , 02110, 02109, 02118 as some of the most expensive zip codes in Boston

# Busiest Time of the year 
## Data Preparation

In [ ]:
BostonCalendar['available'].value_counts()
BostonCalendar['date'].isnull().sum()

In [ ]:
df_avail = BostonCalendar[['listing_id','available','date']]

In [ ]:
df_avail.head()

In [ ]:
# Selecting all the dates on which any listing is available
df_true_available = df_avail.loc[df_avail['available']=='t']

In [ ]:
df_true_available

In [ ]:
most_occupied = pd.DataFrame(df_true_available['date'].value_counts())

In [ ]:
most_occupied['dates'] = most_occupied.index

In [ ]:
most_occupied

### Since we are using the data of a single year so the year part of a date is not significant in this context.

In [ ]:
most_occupied['dates'] = most_occupied['dates'].apply(lambda x: x[5:])

In [ ]:
most_occupied.sort_values(by = ['dates'],inplace = True)

In [ ]:
most_occupied.iloc[0,0].sum()

In [ ]:
month_dict = {}

In [ ]:
month_num = 1
for i in range(0,most_occupied.shape[0],30):
    month_dict["Month"+str(month_num)] = most_occupied.iloc[i:i+30,0].sum()
    month_num += 1
    

## Visualisation

In [ ]:
# plotting the number of availabilities in every week of the year.
month_plot = plt.figure()
month_plot.set_figwidth(40)
month_plot.set_figheight(8)
plt.plot(month_dict.keys(),month_dict.values())
plt.xlabel("Months")
plt.ylabel("Average Room Availability")
plt.tick_params(axis='x', which='major', labelsize=15)
plt.show()

# Analysis

## We can observe in the plot above that in September(Month 9) the availability is least hence we can deduce that it is the busiest time of year and we can increase prices.